In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

In [2]:
#이미지가 저장될 폴더가 없다면 만듭니다.
import os
if not os.path.exists("./gan_images"):
    os.makedirs("./gan_images")

In [3]:
seed=3
np.random.seed(seed)
tf.random.set_seed(seed)

In [4]:
#생성자 모델을 만듭니다.
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))


In [5]:
#판별자 모델을 만듭니다.
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False

In [6]:
#생성자와 판별자 모델을 연결시키는 gan 모델을 만듭니다.
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential (Sequential)      (None, 28, 28, 1)         865281    
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 212865    
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [7]:
#함수없이 하나씩 풀어봅시다.

(X_train, _), (_, _) = mnist.load_data() # 손글씨 데이터의 X_train 부분만 가져옵니다.

In [ ]:
X_train[0]

In [ ]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)

In [8]:
#주피터 노트북 배열 표현, 5개까지 출력하고 나머지 ... 화면전체 길이 110
import numpy as np
np.set_printoptions(edgeitems=5)
np.core.arrayprint._line_width = 110

In [9]:
#실수형으로 변환하였습니다.
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')

In [ ]:
print(X_train[0])

In [10]:
# 픽셀값은 0에서 255사이의 값입니다. 
# 이전에 255로 나누어 줄때는 이를 0~1사이의 값으로 바꾸었습니다.
# -1 ~ 1 사이의 값으로 바꾸기 위해 127.5를 빼준 뒤 127.5로 나누어 줬습니다.
X_train = (X_train - 127.5) / 127.5  

In [ ]:
print(X_train.shape)
#print(Y_train.shape)
#print(X_test.shape)
#print(Y_test.shape)

In [11]:
batch_size = 200
true = np.ones((batch_size, 1))

In [ ]:
true

In [12]:
fake = np.zeros((batch_size, 1))

In [ ]:
fake

In [13]:
idx = np.random.randint(0, X_train.shape[0], batch_size)

In [ ]:
print(idx)
print(len(idx))

In [14]:
imgs = X_train[idx]
imgs

array([[[[-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.]],

        [[-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.]],

        [[-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.]],

        [[-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.]],

        [[-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.]],

        ...,

        [[-1.],
         [-1.],
         [-1.],
         [-1.],
         [-1.],
         ...,
         [-1

In [17]:
d_loss_real = discriminator.train_on_batch(imgs, true)
d_loss_real

0.28550434

0.70585734